In [49]:
!pip install fosforml snowflake  snowflake-connector-python snowflake-snowpark-python snowflake-snowpark-python[pandas] seaborn matplotlib numpy pandas scikit-learn refractio refractml refractio[snowflake]

You should consider upgrading via the 'pip install --upgrade pip' command.


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#energyDataset = pd.read_excel('Energy_Inventory.xlsx')

In [51]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [52]:
table_name = 'INVENTORY_REFINERY'

In [53]:
sf_df = my_session.sql("select * from {}".format(table_name))
energyDataset =  sf_df.to_pandas()
energyDataset

,REFINERY_AREA_DESC,REFINERY_AREA,PLANT_NAME,PLANT_STATUS,MATERIAL_GROUP,MATERIAL_GROUP_DESCRIPTION,MATERIAL_NUMBER,MATERIAL_NUMBER_DESCRIPTION,LOCATION_ID,LOCATION_CODE,PW_MATERIAL_CODE,STOCK_ID,ITEM_TYPE,TARGET_TYPE,TRANSIT_TYPE,TERMINAL_TYPE,INVENTORY_DATE,VOLUME_BBL,VOLUME_MBD,UOM,BOOK_VOLUME_BBL,BOOK_VOLUME_MBD,BOOK_UOM,FACT_REFINERY_PLANT_ID,CITY,FINANCE_CATEGORY,PLAN_TYPE,PLAN_ID,AOP_STATUS,AOP_DESCRIPTION,FLAGACTUAL,FLAGPLAN1,FLAGPLAN2,STOCK_FAMILY,STOCK_CATEGORY_NEW,STOCK_SUB_CATEGORY,LOCATION_LEVEL_0,LOCATION_LEVEL_1,LOCATION_LEVEL_2,LOCATION_LEVEL_3,STOCK_CATEGORY
0,CENTRAL AMERICA,AMR,IMPORTACIONES SPOT TRM,ACTIVE,105100,Not Classified,240133000,Anhydrous Ethanol (CO),478,COLOMBIA,ETHANOL,478,PhysicalInventory,NOT CLASSIFIED,IN TANK,TERMINAL,2021-10-30,0.00,0.00,BBL,148.72,0.15,BBL,3556,OTHER CITIES,Refining Revenue,OTHERS,Not Classified,Not Classified,Not Classified,No,No,No,OTHER,OTHER,ETHANOL,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,ETHANOL
1,CENTRAL AMERICA,CA,BUENAVENTURA TRM,ACTIVE,105100,Not Classified,240133000,Anhydrous Ethanol (CO),478,COLOMBIA,ETHANOL,478,PhysicalInventory,NOT CLASSIFIED,IN TANK,TERMINAL,2021-10-30,0.00,0.00,BBL,608.66,0.61,BBL,3544,OTHER CITIES,Refining Revenue,OTHERS,Not Classified,Not Classified,Not Classified,No,No,No,OTHER,OTHER,ETHANOL,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,ETHANOL
2,CENTRAL AMERICA,CA,CARTAGO TRM,ACTIVE,105100,Not Classified,240133000,Anhydrous Ethanol (CO),478,COLOMBIA,ETHANOL,478,PhysicalInventory,NOT CLASSIFIED,IN TANK,TERMINAL,2021-10-30,0.00,0.00,BBL,276.31,0.28,BBL,3545,OTHER CITIES,Refining Revenue,OTHERS,Not Classified,Not Classified,Not Classified,No,No,No,OTHER,OTHER,ETHANOL,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,ETHANOL
3,CENTRAL AMERICA,CA,GALAPA TRM,ACTIVE,105100,Not Classified,240133000,Anhydrous Ethanol (CO),478,COLOMBIA,ETHANOL,478,PhysicalInventory,NOT CLASSIFIED,IN TANK,TERMINAL,2021-10-30,0.00,0.00,BBL,291.59,0.29,BBL,3532,OTHER CITIES,Refining Revenue,OTHERS,Not Classified,Not Classified,Not Classified,No,No,No,OTHER,OTHER,ETHANOL,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,ETHANOL
4,CENTRAL AMERICA,CA,GUALANDAY TRM,ACTIVE,105100,Not Classified,240133000,Anhydrous Ethanol (CO),478,COLOMBIA,ETHANOL,478,PhysicalInventory,NOT CLASSIFIED,IN TANK,TERMINAL,2021-10-30,0.00,0.00,BBL,446.33,0.45,BBL,3542,OTHER CITIES,Refining Revenue,OTHERS,Not Classified,Not Classified,Not Classified,No,No,No,OTHER,OTHER,ETHANOL,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,ETHANOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134139,CENTRAL AMERICA,SV,ACAJUTLA TRM,ACTIVE,Not Classified,Not Classified,Not Classified,Not Classified,435,ACAJUTLA,Not Classified,491,TargetInventory,CALCULATED,IN TANK,TERMINAL,2024-12-31,59062.65,59.06,MB,59062.65,59.06,MB,3558,ACAJUTLA,Refining Revenue,NOT CLASSIFIED,Not Classified,Not Classified,Not Classified,No,No,No,PRODUCTS,GASOLINE,PUL,US EAST,LATAM SYSTEM,WCENTRAL AMERICA,ACAJUTLA,MOGAS
134140,CENTRAL AMERICA,SV,ACAJUTLA TRM,ACTIVE,Not Classified,Not Classified,Not Classified,Not Classified,435,ACAJUTLA,Not Classified,111,TargetInventory,CALCULATED,IN TANK,TERMINAL,2024-12-31,61617.37,61.62,MB,61617.37,61.62,MB,3558,ACAJUTLA,Refining Revenue,NOT CLASSIFIED,Not Classified,Not Classified,Not Classified,No,No,No,PRODUCTS,GASOLINE,RUL,US EAST,LATAM SYSTEM,WCENTRAL AMERICA,ACAJUTLA,MOGAS
134141,CENTRAL AMERICA,SV,ACAJUTLA TRM,ACTIVE,Not Classified,Not Classified,Not Classified,Not Classified,435,ACAJUTLA,Not Classified,839,TargetInventory,CALCULATED,IN TANK,TERMINAL,2024-12-31,89187.03,89.19,MB,89187.03,89.19,MB,3558,ACAJUTLA,Refining Revenue,NOT CLASSIFIED,Not Classified,Not Classified,Not Classified,No,No,No,PRODUCTS,DISTILLATE,LSD,US EAST,LATAM SYSTEM,WCENTRAL AMERICA,ACAJUTLA,LSD
134142,CENTRAL AMERICA,SV,RASA TRM,ACTIVE,450000,Not Classified,940033000,BUNKER F.OIL (GT/HN/PA/SV),852,RASA,HSFO,557,PhysicalInventory,NOT CLASSIFIE

In [54]:
actualVolumes = energyDataset[energyDataset['FLAGACTUAL'] == 'Yes']
targetVolumes = energyDataset[energyDataset['ITEM_TYPE'] == 'TargetInventory']
print(len(actualVolumes), len(targetVolumes))

38558 34905


In [55]:
uniqueCombinations = actualVolumes.drop_duplicates(subset=['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY','STOCK_SUB_CATEGORY','INVENTORY_DATE','PLANT_NAME'])

# Compare lengths
if len(actualVolumes) == len(uniqueCombinations):
    print("All records are unique based on the combination of the six columns.")
else:
    print("There are duplicate records based on the combination of the six columns.")
print(len(actualVolumes) , len(uniqueCombinations))

There are duplicate records based on the combination of the six columns.
38558 27770


In [56]:
uniqueCombinations = targetVolumes.drop_duplicates(subset=['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY','STOCK_SUB_CATEGORY','INVENTORY_DATE','PLANT_NAME'])

# Compare lengths
if len(targetVolumes) == len(uniqueCombinations):
    print("All records are unique based on the combination of the six columns.")
else:
    print("There are duplicate records based on the combination of the six columns.")
print(len(targetVolumes) , len(uniqueCombinations))

There are duplicate records based on the combination of the six columns.
34905 34827


In [57]:
groupedActual = actualVolumes.groupby(['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY','STOCK_SUB_CATEGORY','INVENTORY_DATE','PLANT_NAME'])[['VOLUME_MBD']].sum().reset_index()
print(len(actualVolumes) , len(groupedActual))

38558 27770


In [58]:
groupedTarget = targetVolumes.groupby(['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY','STOCK_SUB_CATEGORY','INVENTORY_DATE','PLANT_NAME','TARGET_TYPE','ITEM_TYPE','TRANSIT_TYPE','TERMINAL_TYPE','REFINERY_AREA_DESC','LOCATION_ID','LOCATION_LEVEL_0','LOCATION_LEVEL_1','LOCATION_LEVEL_2','LOCATION_LEVEL_3','REFINERY_AREA','PLANT_STATUS'])[['VOLUME_MBD']].sum().reset_index()
print(len(targetVolumes) , len(groupedTarget))

34905 34827


In [59]:
groupedActual = groupedActual.rename(columns={'VOLUME_MBD':'VOLUME_MBD_ACTUAL'})

groupedTarget = groupedTarget.rename(columns={'VOLUME_MBD':'VOLUME_MBD_TARGET'})


In [60]:
merged_df = pd.merge(groupedTarget, groupedActual , on=['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY','STOCK_SUB_CATEGORY','INVENTORY_DATE','PLANT_NAME'], how='left')
pd.set_option('display.max_columns', None)
merged_df

,STOCK_FAMILY,STOCK_CATEGORY_NEW,STOCK_CATEGORY,STOCK_SUB_CATEGORY,INVENTORY_DATE,PLANT_NAME,TARGET_TYPE,ITEM_TYPE,TRANSIT_TYPE,TERMINAL_TYPE,REFINERY_AREA_DESC,LOCATION_ID,LOCATION_LEVEL_0,LOCATION_LEVEL_1,LOCATION_LEVEL_2,LOCATION_LEVEL_3,REFINERY_AREA,PLANT_STATUS,VOLUME_MBD_TARGET,VOLUME_MBD_ACTUAL
0,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,EL SEGUNDO CA RFY,DEFINED,TargetInventory,IN TANK,REFINERY,EL SEGUNDO,372,US WEST COAST,EL SEGUNDO SUPPLY AREA,EL SEGUNDO REFINERY,EL SEGUNDO,ELSE,ACTIVE,2345.00,2162.75
1,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,NaN
2,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,PASADENA TX RFY,DEFINED,TargetInventory,IN TANK,REFINERY,PASADENA,808,US EAST,PASADENA SUPPLY AREA,PASADENA REFINERY,PASADENA,PASA,ACTIVE,745.00,1325.86
3,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,PASCAGOULA MS RFY,DEFINED,TargetInventory,IN TANK,REFINERY,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,2186.00,1920.00
4,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,RICHMOND CA RFY,DEFINED,TargetInventory,IN TANK,REFINERY,RICHMOND,315,US WEST COAST,RICHMOND SUPPLY AREA,BAY AREA,BAPL,RICH,ACTIVE,1780.00,2162.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34822,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WASKOM TX TRM SUNOCO,CALCULATED,TargetInventory,IN TANK,TERMINAL,PASCAGOULA,848,US EAST,PASCAGOULA SUPPLY AREA,FLORIDA,PANAMA CITY,PASC,ACTIVE,4.23,NaN
34823,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WATSON PPL CA INTRANSIT,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,EL SEGUNDO,797,US WEST COAST,EL SEGUNDO SUPPLY AREA,SOCAL AREA,KM SOCAL,ELSE,ACTIVE,107.85,NaN
34824,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WILLIAMS PPL OK INTRANSIT,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,PASCAGOULA,848,US EAST,PASCAGOULA SUPPLY AREA,FLORIDA,PANAMA CITY,PASC,ACTIVE,2.20,NaN
34825,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,YUMBO - BUENAVENTURA PPL,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,CENTRAL AMERICA,478,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,CA,ACTIVE,8.48,NaN


In [61]:
import hashlib
def hashColumns(row):
    concatenated = f"{row['STOCK_FAMILY']}{row['STOCK_CATEGORY_NEW']}{row['STOCK_CATEGORY']}{row['STOCK_SUB_CATEGORY']}{row['INVENTORY_DATE']}{row['PLANT_NAME']}"
    return hashlib.sha256(concatenated.encode()).hexdigest()

merged_df['RECORD_ID'] = merged_df.apply(hashColumns, axis=1)
future_data = merged_df

In [62]:
# Check for missing values
missing_values = merged_df.isnull()
#print(missing_values)

# Count missing values in each column
missing_count = merged_df.isnull().sum()
print(missing_count)

STOCK_FAMILY             0
STOCK_CATEGORY_NEW       0
STOCK_CATEGORY           0
STOCK_SUB_CATEGORY       0
INVENTORY_DATE           0
PLANT_NAME               0
TARGET_TYPE              0
ITEM_TYPE                0
TRANSIT_TYPE             0
TERMINAL_TYPE            0
REFINERY_AREA_DESC       0
LOCATION_ID              0
LOCATION_LEVEL_0         0
LOCATION_LEVEL_1         0
LOCATION_LEVEL_2         0
LOCATION_LEVEL_3         0
REFINERY_AREA            0
PLANT_STATUS             0
VOLUME_MBD_TARGET        0
VOLUME_MBD_ACTUAL     7832
RECORD_ID                0
dtype: int64


In [63]:
unique_values = merged_df.nunique()
print(unique_values)

STOCK_FAMILY              4
STOCK_CATEGORY_NEW        9
STOCK_CATEGORY           42
STOCK_SUB_CATEGORY       46
INVENTORY_DATE           39
PLANT_NAME              235
TARGET_TYPE               2
ITEM_TYPE                 1
TRANSIT_TYPE              2
TERMINAL_TYPE             3
REFINERY_AREA_DESC        6
LOCATION_ID              41
LOCATION_LEVEL_0          3
LOCATION_LEVEL_1         10
LOCATION_LEVEL_2         30
LOCATION_LEVEL_3         41
REFINERY_AREA            14
PLANT_STATUS              1
VOLUME_MBD_TARGET       625
VOLUME_MBD_ACTUAL      9074
RECORD_ID             34827
dtype: int64


In [64]:
merged_df['INVENTORY_DATE'] = pd.to_datetime(merged_df['INVENTORY_DATE'])

# Extract year and month
merged_df['YEAR'] = merged_df['INVENTORY_DATE'].dt.year
merged_df['MONTH'] = merged_df['INVENTORY_DATE'].dt.strftime('%b')

In [65]:
merged_df.drop(['STOCK_SUB_CATEGORY','INVENTORY_DATE','PLANT_NAME','LOCATION_ID','LOCATION_LEVEL_3','ITEM_TYPE','PLANT_STATUS'],axis=1, inplace=True)

In [66]:
unique_values = merged_df.nunique()
print(unique_values)

STOCK_FAMILY              4
STOCK_CATEGORY_NEW        9
STOCK_CATEGORY           42
TARGET_TYPE               2
TRANSIT_TYPE              2
TERMINAL_TYPE             3
REFINERY_AREA_DESC        6
LOCATION_LEVEL_0          3
LOCATION_LEVEL_1         10
LOCATION_LEVEL_2         30
REFINERY_AREA            14
VOLUME_MBD_TARGET       625
VOLUME_MBD_ACTUAL      9074
RECORD_ID             34827
YEAR                      4
MONTH                    12
dtype: int64


In [67]:
final_dataset = merged_df

In [68]:
one_hot = pd.get_dummies(final_dataset, columns=['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY',
                                           'TRANSIT_TYPE','TERMINAL_TYPE',
                                           'REFINERY_AREA_DESC','LOCATION_LEVEL_0',
                                           'LOCATION_LEVEL_1','LOCATION_LEVEL_2','TARGET_TYPE',
                                           'REFINERY_AREA','YEAR','MONTH'], 
                                            prefix = ['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY',
                                           'TRANSIT_TYPE','TERMINAL_TYPE',
                                           'REFINERY_AREA_DESC','LOCATION_LEVEL_0',
                                           'LOCATION_LEVEL_1','LOCATION_LEVEL_2','TARGET_TYPE',
                                           'REFINERY_AREA','YEAR','MONTH'],drop_first=True, dtype=int)
# concatenate the one-hot encoding with the original dataframe
one_hot = one_hot.drop(['VOLUME_MBD_TARGET','VOLUME_MBD_ACTUAL','RECORD_ID'], axis= 1)
df1 = pd.concat([final_dataset, one_hot], axis=1)
# drop the original 'color' column
df1 = df1.drop(['STOCK_FAMILY', 'STOCK_CATEGORY_NEW', 'STOCK_CATEGORY',
                                           'TRANSIT_TYPE','TERMINAL_TYPE',
                                           'REFINERY_AREA_DESC','LOCATION_LEVEL_0',
                                           'LOCATION_LEVEL_1','LOCATION_LEVEL_2','TARGET_TYPE',
                                           'REFINERY_AREA','YEAR','MONTH'], axis=1)

In [69]:
df1.columns

Index(['VOLUME_MBD_TARGET', 'VOLUME_MBD_ACTUAL', 'RECORD_ID',
       'STOCK_FAMILY_INTERMEDIATES', 'STOCK_FAMILY_OTHER',
       'STOCK_FAMILY_PRODUCTS', 'STOCK_CATEGORY_NEW_CYCLE OIL',
       'STOCK_CATEGORY_NEW_DISTILLATE',
       'STOCK_CATEGORY_NEW_DISTILLATE FEEDSTOCKS',
       'STOCK_CATEGORY_NEW_FUEL OIL FEEDSTOCKS',
       ...
       'MONTH_Dec', 'MONTH_Feb', 'MONTH_Jan', 'MONTH_Jul', 'MONTH_Jun',
       'MONTH_Mar', 'MONTH_May', 'MONTH_Nov', 'MONTH_Oct', 'MONTH_Sep'],
      dtype='object', length=131)

In [70]:
df1.columns = df1.columns.str.replace(' ', '_')
df1.columns = df1.columns.str.replace('_/', '')
df1.columns = df1.columns.str.upper()
df1

,VOLUME_MBD_TARGET,VOLUME_MBD_ACTUAL,RECORD_ID,STOCK_FAMILY_INTERMEDIATES,STOCK_FAMILY_OTHER,STOCK_FAMILY_PRODUCTS,STOCK_CATEGORY_NEW_CYCLE_OIL,STOCK_CATEGORY_NEW_DISTILLATE,STOCK_CATEGORY_NEW_DISTILLATE_FEEDSTOCKS,STOCK_CATEGORY_NEW_FUEL_OIL_FEEDSTOCKS,STOCK_CATEGORY_NEW_GASOIL_FEEDSTOCKS,STOCK_CATEGORY_NEW_GASOLINE,STOCK_CATEGORY_NEW_MOGAS_FEEDSTOCKS,STOCK_CATEGORY_NEW_OTHER,STOCK_CATEGORY_ASPHALT,STOCK_CATEGORY_AVGAS,STOCK_CATEGORY_CHDN_FEED,STOCK_CATEGORY_COKE,STOCK_CATEGORY_CRUDE,STOCK_CATEGORY_DIESEL,STOCK_CATEGORY_DIESEL_ADDITIVES,STOCK_CATEGORY_DIESEL_BLENDSTOCK,STOCK_CATEGORY_DLCO,STOCK_CATEGORY_ETHANOL,STOCK_CATEGORY_FINISHED_BASE_OIL,STOCK_CATEGORY_FINISHED_CHEMICALS,STOCK_CATEGORY_FUEL_OIL,STOCK_CATEGORY_GASOLINE_ADDITIVES,STOCK_CATEGORY_HCGO,STOCK_CATEGORY_HCO,STOCK_CATEGORY_HSR,STOCK_CATEGORY_HSVGO,STOCK_CATEGORY_ISOMAX_TKN_FEED,STOCK_CATEGORY_JET,STOCK_CATEGORY_JET_BLENDSTOCK,STOCK_CATEGORY_LCO,STOCK_CATEGORY_LPG,STOCK_CATEGORY_LSD,STOCK_CATEGORY_LSVGO,STOCK_CATEGORY_LSWR,STOCK_CATEGORY_MOGAS,STOCK_CATEGORY_NAPHTHA,STOCK_CATEGORY_NATURAL_GAS,STOCK_CATEGORY_OTHER,STOCK_CATEGORY_PBOP_FEED,STOCK_CATEGORY_PENHEX,STOCK_CATEGORY_RDS_FEED,STOCK_CATEGORY_RECOVERED_OIL,STOCK_CATEGORY_RESID,STOCK_CATEGORY_SR_DIESEL,STOCK_CATEGORY_SR_JET,STOCK_CATEGORY_SULFUR,STOCK_CATEGORY_TETRAMER,STOCK_CATEGORY_TRANSMIX,STOCK_CATEGORY_WAXY_BASE_OIL,TRANSIT_TYPE_IN_TRANSIT,TERMINAL_TYPE_REFINERY,TERMINAL_TYPE_TERMINAL,REFINERY_AREA_DESC_EL_SEGUNDO,REFINERY_AREA_DESC_PASADENA,REFINERY_AREA_DESC_PASCAGOULA,REFINERY_AREA_DESC_RICHMOND,REFINERY_AREA_DESC_SALT_LAKE,LOCATION_LEVEL_0_US_EAST,LOCATION_LEVEL_0_US_WEST_COAST,LOCATION_LEVEL_1_IDAHO_TERMINALS,LOCATION_LEVEL_1_LATAM_SYSTEM,LOCATION_LEVEL_1_PASADENA_SUPPLY_AREA,LOCATION_LEVEL_1_PASCAGOULA_SUPPLY_AREA,LOCATION_LEVEL_1_PNW_TERMINALS,LOCATION_LEVEL_1_RICHMOND_SUPPLY_AREA,LOCATION_LEVEL_1_SALT_LAKE_REFINERY,LOCATION_LEVEL_1_SALT_LAKE_TERMINALS,LOCATION_LEVEL_1_SOUTH_TERMINALS,LOCATION_LEVEL_2_BOISE_TERMINAL,LOCATION_LEVEL_2_BURLEY_TERMINAL,LOCATION_LEVEL_2_CEDARCTY,LOCATION_LEVEL_2_COLOMBIA,LOCATION_LEVEL_2_COLONIAL,LOCATION_LEVEL_2_ECENTRAL_AMERICA,LOCATION_LEVEL_2_EL_PASO,LOCATION_LEVEL_2_EL_SEGUNDO_REFINERY,LOCATION_LEVEL_2_FLORIDA,LOCATION_LEVEL_2_LA_MS_OH_TN,LOCATION_LEVEL_2_LVHOLLY,LOCATION_LEVEL_2_NY_HARBOR,LOCATION_LEVEL_2_PANAMA,LOCATION_LEVEL_2_PASADENA_REFINERY,LOCATION_LEVEL_2_PASC_RACK_FOB,LOCATION_LEVEL_2_PASCAGOULA_REFINERY,LOCATION_LEVEL_2_PASCO,LOCATION_LEVEL_2_PASCO_PPL,LOCATION_LEVEL_2_PHOENIX_AREA,LOCATION_LEVEL_2_PLANTATION_DUAL,LOCATION_LEVEL_2_POCATELLO_TERMINAL,LOCATION_LEVEL_2_POCATELLO_UPRR,LOCATION_LEVEL_2_SALT_LAKE_PPL_INTRANSIT,LOCATION_LEVEL_2_SALT_LAKE_REFINERY,LOCATION_LEVEL_2_SALT_LAKE_TERMINAL,LOCATION_LEVEL_2_SOCAL_AREA,LOCATION_LEVEL_2_SPOKANE_HOLLY,LOCATION_LEVEL_2_TEXAS,LOCATION_LEVEL_2_WCENTRAL_AMERICA,TARGET_TYPE_DEFINED,REFINERY_AREA_CA,REFINERY_AREA_ELPA,REFINERY_AREA_ELSE,REFINERY_AREA_GT,REFINERY_AREA_HN,REFINERY_AREA_HNOA,REFINERY_AREA_PA,REFINERY_AREA_PAOA,REFINERY_AREA_PASA,REFINERY_AREA_PASC,REFINERY_AREA_RICH,REFINERY_AREA_SLAK,REFINERY_AREA_SV,YEAR_2022,YEAR_2023,YEAR_2024,MONTH_AUG,MONTH_DEC,MONTH_FEB,MONTH_JAN,MONTH_JUL,MONTH_JUN,MONTH_MAR,MONTH_MAY,MONTH_NOV,MONTH_OCT,MONTH_SEP
0,2345.00,2162.75,6ed824442bb810cf83cdfc4a4812ce22e2f270e60d0459...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,78.82,NaN,e7c5a8a2b253d2d195f32439f4e7b9259cc2c8b5f47a07...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,745.00,1325.86,28fd622243b93381329a5f439356de50f15a428e1d00c3...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [71]:

X_array = df1.drop(['VOLUME_MBD_ACTUAL'], axis= 1)
y_array= df1[['VOLUME_MBD_ACTUAL']]
X = X_array.iloc[:].values
y = y_array.iloc[:].values
print(X)

[[2345.0
  '6ed824442bb810cf83cdfc4a4812ce22e2f270e60d0459e8f819a5e95e245229' 0
  ... 0 1 0]
 [78.82
  'e7c5a8a2b253d2d195f32439f4e7b9259cc2c8b5f47a07ca9df7dc406d59cfc6' 0
  ... 0 1 0]
 [745.0
  '28fd622243b93381329a5f439356de50f15a428e1d00c321ac0bbc08b0d0088d' 0
  ... 0 1 0]
 ...
 [2.2 'd85f3d9c555d3645f8cfc663bc5b37b0095d48d0ee384ebf0c9a8cbf6d3941ad'
  0 ... 0 0 0]
 [8.48 '8d0e2dc85623a47298eb31cd44f078c0e498fdde03c0c01fbef1e7117586705a'
  0 ... 0 0 0]
 [10.79
  '059a620758b7a07f94fdd9e6155d2e39a47906f360eea734155382c62d5603bc' 0
  ... 0 0 0]]


In [72]:
future_input_data = df1[df1['VOLUME_MBD_ACTUAL'].isnull()]
future_input_data

,VOLUME_MBD_TARGET,VOLUME_MBD_ACTUAL,RECORD_ID,STOCK_FAMILY_INTERMEDIATES,STOCK_FAMILY_OTHER,STOCK_FAMILY_PRODUCTS,STOCK_CATEGORY_NEW_CYCLE_OIL,STOCK_CATEGORY_NEW_DISTILLATE,STOCK_CATEGORY_NEW_DISTILLATE_FEEDSTOCKS,STOCK_CATEGORY_NEW_FUEL_OIL_FEEDSTOCKS,STOCK_CATEGORY_NEW_GASOIL_FEEDSTOCKS,STOCK_CATEGORY_NEW_GASOLINE,STOCK_CATEGORY_NEW_MOGAS_FEEDSTOCKS,STOCK_CATEGORY_NEW_OTHER,STOCK_CATEGORY_ASPHALT,STOCK_CATEGORY_AVGAS,STOCK_CATEGORY_CHDN_FEED,STOCK_CATEGORY_COKE,STOCK_CATEGORY_CRUDE,STOCK_CATEGORY_DIESEL,STOCK_CATEGORY_DIESEL_ADDITIVES,STOCK_CATEGORY_DIESEL_BLENDSTOCK,STOCK_CATEGORY_DLCO,STOCK_CATEGORY_ETHANOL,STOCK_CATEGORY_FINISHED_BASE_OIL,STOCK_CATEGORY_FINISHED_CHEMICALS,STOCK_CATEGORY_FUEL_OIL,STOCK_CATEGORY_GASOLINE_ADDITIVES,STOCK_CATEGORY_HCGO,STOCK_CATEGORY_HCO,STOCK_CATEGORY_HSR,STOCK_CATEGORY_HSVGO,STOCK_CATEGORY_ISOMAX_TKN_FEED,STOCK_CATEGORY_JET,STOCK_CATEGORY_JET_BLENDSTOCK,STOCK_CATEGORY_LCO,STOCK_CATEGORY_LPG,STOCK_CATEGORY_LSD,STOCK_CATEGORY_LSVGO,STOCK_CATEGORY_LSWR,STOCK_CATEGORY_MOGAS,STOCK_CATEGORY_NAPHTHA,STOCK_CATEGORY_NATURAL_GAS,STOCK_CATEGORY_OTHER,STOCK_CATEGORY_PBOP_FEED,STOCK_CATEGORY_PENHEX,STOCK_CATEGORY_RDS_FEED,STOCK_CATEGORY_RECOVERED_OIL,STOCK_CATEGORY_RESID,STOCK_CATEGORY_SR_DIESEL,STOCK_CATEGORY_SR_JET,STOCK_CATEGORY_SULFUR,STOCK_CATEGORY_TETRAMER,STOCK_CATEGORY_TRANSMIX,STOCK_CATEGORY_WAXY_BASE_OIL,TRANSIT_TYPE_IN_TRANSIT,TERMINAL_TYPE_REFINERY,TERMINAL_TYPE_TERMINAL,REFINERY_AREA_DESC_EL_SEGUNDO,REFINERY_AREA_DESC_PASADENA,REFINERY_AREA_DESC_PASCAGOULA,REFINERY_AREA_DESC_RICHMOND,REFINERY_AREA_DESC_SALT_LAKE,LOCATION_LEVEL_0_US_EAST,LOCATION_LEVEL_0_US_WEST_COAST,LOCATION_LEVEL_1_IDAHO_TERMINALS,LOCATION_LEVEL_1_LATAM_SYSTEM,LOCATION_LEVEL_1_PASADENA_SUPPLY_AREA,LOCATION_LEVEL_1_PASCAGOULA_SUPPLY_AREA,LOCATION_LEVEL_1_PNW_TERMINALS,LOCATION_LEVEL_1_RICHMOND_SUPPLY_AREA,LOCATION_LEVEL_1_SALT_LAKE_REFINERY,LOCATION_LEVEL_1_SALT_LAKE_TERMINALS,LOCATION_LEVEL_1_SOUTH_TERMINALS,LOCATION_LEVEL_2_BOISE_TERMINAL,LOCATION_LEVEL_2_BURLEY_TERMINAL,LOCATION_LEVEL_2_CEDARCTY,LOCATION_LEVEL_2_COLOMBIA,LOCATION_LEVEL_2_COLONIAL,LOCATION_LEVEL_2_ECENTRAL_AMERICA,LOCATION_LEVEL_2_EL_PASO,LOCATION_LEVEL_2_EL_SEGUNDO_REFINERY,LOCATION_LEVEL_2_FLORIDA,LOCATION_LEVEL_2_LA_MS_OH_TN,LOCATION_LEVEL_2_LVHOLLY,LOCATION_LEVEL_2_NY_HARBOR,LOCATION_LEVEL_2_PANAMA,LOCATION_LEVEL_2_PASADENA_REFINERY,LOCATION_LEVEL_2_PASC_RACK_FOB,LOCATION_LEVEL_2_PASCAGOULA_REFINERY,LOCATION_LEVEL_2_PASCO,LOCATION_LEVEL_2_PASCO_PPL,LOCATION_LEVEL_2_PHOENIX_AREA,LOCATION_LEVEL_2_PLANTATION_DUAL,LOCATION_LEVEL_2_POCATELLO_TERMINAL,LOCATION_LEVEL_2_POCATELLO_UPRR,LOCATION_LEVEL_2_SALT_LAKE_PPL_INTRANSIT,LOCATION_LEVEL_2_SALT_LAKE_REFINERY,LOCATION_LEVEL_2_SALT_LAKE_TERMINAL,LOCATION_LEVEL_2_SOCAL_AREA,LOCATION_LEVEL_2_SPOKANE_HOLLY,LOCATION_LEVEL_2_TEXAS,LOCATION_LEVEL_2_WCENTRAL_AMERICA,TARGET_TYPE_DEFINED,REFINERY_AREA_CA,REFINERY_AREA_ELPA,REFINERY_AREA_ELSE,REFINERY_AREA_GT,REFINERY_AREA_HN,REFINERY_AREA_HNOA,REFINERY_AREA_PA,REFINERY_AREA_PAOA,REFINERY_AREA_PASA,REFINERY_AREA_PASC,REFINERY_AREA_RICH,REFINERY_AREA_SLAK,REFINERY_AREA_SV,YEAR_2022,YEAR_2023,YEAR_2024,MONTH_AUG,MONTH_DEC,MONTH_FEB,MONTH_JAN,MONTH_JUL,MONTH_JUN,MONTH_MAR,MONTH_MAY,MONTH_NOV,MONTH_OCT,MONTH_SEP
1,78.82,NaN,e7c5a8a2b253d2d195f32439f4e7b9259cc2c8b5f47a07...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,78.82,NaN,dfc68bbc395067134d5d6e54be9352a65b0d6ba38e2d53...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
13,78.82,NaN,4459498978e9508cf0dc4071eeb414248aad4efdc2110a...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [73]:
training_data = df1[df1['VOLUME_MBD_ACTUAL'].notnull()]

In [74]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(training_data.drop(['VOLUME_MBD_ACTUAL','RECORD_ID'], axis=1), training_data['VOLUME_MBD_ACTUAL'], test_size=0.2, random_state=42)

In [75]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [76]:
y_pred = regressor.predict(X_test)


In [ ]:
results_df = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R2'])

models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    KNeighborsRegressor(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    SVR()
]
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #results_df = results_df.append({'Model': model_name, 'MSE': mse, 'MAE': mae, 'R2':r2}, ignore_index=True)
    results_df = pd.concat([results_df, pd.DataFrame([{'Model': model_name, 'MSE': mse, 'MAE': mae, 'R2':r2}])], ignore_index=True)
print(results_df)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

neighbors=np.arange(1,9)
train_accuracy=np.empty(len(neighbors))
test_accuracy=np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    knn=KNeighborsRegressor(n_neighbors=k)
    
    knn.fit(X_train, y_train)
    
    train_accuracy[i]=knn.score(X_train, y_train)
    
    test_accuracy[i]=knn.score(X_test, y_test)
print(train_accuracy)
print(test_accuracy)

In [ ]:
knn = KNeighborsRegressor()

# Define the parameter grid for the grid search
param_grid = {
    'n_neighbors': [7,9,11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by the grid search
print("Best parameters: ", grid_search.best_params_)

# Print the best score found by the grid search
print("Best score: ", -grid_search.best_score_)

# Use the best estimator to make predictions on the test set
best_knn = grid_search.best_estimator_
predictions = best_knn.predict(X_test)

In [ ]:
y_pred = best_knn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
knn_score = best_knn.score(X_test, y_test)
print(mse)
print(rmse)
print(r2)
print(knn_score)

In [ ]:
y_pred_original = np.expm1(y_pred)
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')
plt.show()

In [ ]:
# param_grid = {
#     'n_estimators': [100,200],
#     'learning_rate': [0.2],
#     'max_depth': [4],
#     'min_samples_split': [10],
#     'min_samples_leaf': [ 4],
#     'subsample': [1.0]
# }
# gb_reg = GradientBoostingRegressor()
# grid_search = GridSearchCV(estimator=gb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
# grid_search.fit(X_train, y_train)
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_
# print(best_params, best_model)

In [ ]:
# y_pred = grid_search.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)

In [ ]:
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print("Mean Squared Error (MSE):", mse)
# print("Mean Absolute Error (MAE):", mae)
# print("R-squared (R2):", r2)

In [ ]:
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': best_model.feature_importances_})
# feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
# print("Feature Importance:")
# print(feature_importance)

In [ ]:
y_pred_df = pd.DataFrame(y_pred , columns = ['volume_pred'])
y_pred_df

In [45]:
from fosforml import register_model

registered_model = register_model(
  model_obj=best_knn,
  session=my_session,
  x_train=X_train,
  y_train=y_train,
  x_test=X_test,
  y_test=y_test,
  y_pred=y_pred_df,
  source="Notebook",
  dataset_name="REFINERY_INVENTORY",
  dataset_source="Snowflake",
  name="REFINERY_INVENTORY",
  description="This model predict actual volume of oil and gas generated by a plant",
  flavour="sklearn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████████                                                     25.0%


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [43]:
X_test.to_csv('Testing_data.csv')

In [46]:
!pip install fosforml snowflake  snowflake-connector-python snowflake-snowpark-python snowflake-snowpark-python[pandas] seaborn matplotlib numpy pandas scikit-learn refractio refractml refractio[snowflake]

     |████████████████████████████████| 296kB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 9.9MB/s  eta 0:00:01
     |████████████████████████████████| 665kB 36.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 20.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 18.9MB/s eta 0:00:01
     |████████████████████████████████| 440kB 100.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 17.0MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 98.7MB/s eta 0:00:01
  Created wheel for shutils: filename=shutils-0.1.0-cp39-none-any.whl size=3274 sha256=80e3c194245a335abcdc01620dba1695f73a7164387bc1027b3f6dd294cd343d
  Stored in directory: /home/mosaic-ai/.cache/pip/wheels/ab/d0/0e/613976a1b51b5654859e2a82ade64329859bce431e280f2a39
Successfully built shutils
ERROR: refractio 2.1.5.6 has requirement pandas==2.0.0, but you'll have pandas 2.2.0 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; 

In [47]:
type(registered_model)

str

In [48]:
registered_model.dir()

AttributeError: 'str' object has no attribute 'dir'

In [83]:
original_future_data =  future_data[future_data['VOLUME_MBD_ACTUAL'].isnull()]
original_future_data

,STOCK_FAMILY,STOCK_CATEGORY_NEW,STOCK_CATEGORY,STOCK_SUB_CATEGORY,INVENTORY_DATE,PLANT_NAME,TARGET_TYPE,ITEM_TYPE,TRANSIT_TYPE,TERMINAL_TYPE,REFINERY_AREA_DESC,LOCATION_ID,LOCATION_LEVEL_0,LOCATION_LEVEL_1,LOCATION_LEVEL_2,LOCATION_LEVEL_3,REFINERY_AREA,PLANT_STATUS,VOLUME_MBD_TARGET,VOLUME_MBD_ACTUAL,RECORD_ID
1,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,NaN,e7c5a8a2b253d2d195f32439f4e7b9259cc2c8b5f47a07...
7,CRUDE,CRUDE,CRUDE,CRUDE,2021-11-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,NaN,dfc68bbc395067134d5d6e54be9352a65b0d6ba38e2d53...
13,CRUDE,CRUDE,CRUDE,CRUDE,2021-12-31,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,NaN,4459498978e9508cf0dc4071eeb414248aad4efdc2110a...
19,CRUDE,CRUDE,CRUDE,CRUDE,2022-01-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,NaN,6501692fba7eddfaec805e76bb486092f0003a6015cfb9...
25,CRUDE,CRUDE,CRUDE,CRUDE,2022-02-28,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,NaN,c53d8bd441c8dea12bf2bd575699b3f3185107aac76cf5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34822,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WASKOM TX TRM SUNOCO,CALCULATED,TargetInventory,IN TANK,TERMINAL,PASCAGOULA,848,US EAST,PASCAGOULA SUPPLY AREA,FLORIDA,PANAMA CITY,PASC,ACTIVE,4.23,NaN,a85bca20d670421cc88c63d4132157a55aeed2443ad166...
34823,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WATSON PPL CA INTRANSIT,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,EL SEGUNDO,797,US WEST COAST,EL SEGUNDO SUPPLY AREA,SOCAL AREA,KM SOCAL,ELSE,ACTIVE,107.85,NaN,0ab1963368e77af69d8fe7fc0a6d4ece7cd8513d942966...
34824,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WILLIAMS PPL OK INTRANSIT,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,PASCAGOULA,848,US EAST,PASCAGOULA SUPPLY AREA,FLORIDA,PANAMA CITY,PASC,ACTIVE,2.20,NaN,d85f3d9c555d3645f8cfc663bc5b37b0095d48d0ee384e...
34825,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,YUMBO - BUENAVENTURA PPL,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,CENTRAL AMERICA,478,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,CA,ACTIVE,8.48,NaN,8d0e2dc85623a47298eb31cd44f078c0e498fdde03c0c0...


In [84]:
future_input_data

,VOLUME_MBD_TARGET,VOLUME_MBD_ACTUAL,RECORD_ID,STOCK_FAMILY_INTERMEDIATES,STOCK_FAMILY_OTHER,STOCK_FAMILY_PRODUCTS,STOCK_CATEGORY_NEW_CYCLE_OIL,STOCK_CATEGORY_NEW_DISTILLATE,STOCK_CATEGORY_NEW_DISTILLATE_FEEDSTOCKS,STOCK_CATEGORY_NEW_FUEL_OIL_FEEDSTOCKS,STOCK_CATEGORY_NEW_GASOIL_FEEDSTOCKS,STOCK_CATEGORY_NEW_GASOLINE,STOCK_CATEGORY_NEW_MOGAS_FEEDSTOCKS,STOCK_CATEGORY_NEW_OTHER,STOCK_CATEGORY_ASPHALT,STOCK_CATEGORY_AVGAS,STOCK_CATEGORY_CHDN_FEED,STOCK_CATEGORY_COKE,STOCK_CATEGORY_CRUDE,STOCK_CATEGORY_DIESEL,STOCK_CATEGORY_DIESEL_ADDITIVES,STOCK_CATEGORY_DIESEL_BLENDSTOCK,STOCK_CATEGORY_DLCO,STOCK_CATEGORY_ETHANOL,STOCK_CATEGORY_FINISHED_BASE_OIL,STOCK_CATEGORY_FINISHED_CHEMICALS,STOCK_CATEGORY_FUEL_OIL,STOCK_CATEGORY_GASOLINE_ADDITIVES,STOCK_CATEGORY_HCGO,STOCK_CATEGORY_HCO,STOCK_CATEGORY_HSR,STOCK_CATEGORY_HSVGO,STOCK_CATEGORY_ISOMAX_TKN_FEED,STOCK_CATEGORY_JET,STOCK_CATEGORY_JET_BLENDSTOCK,STOCK_CATEGORY_LCO,STOCK_CATEGORY_LPG,STOCK_CATEGORY_LSD,STOCK_CATEGORY_LSVGO,STOCK_CATEGORY_LSWR,STOCK_CATEGORY_MOGAS,STOCK_CATEGORY_NAPHTHA,STOCK_CATEGORY_NATURAL_GAS,STOCK_CATEGORY_OTHER,STOCK_CATEGORY_PBOP_FEED,STOCK_CATEGORY_PENHEX,STOCK_CATEGORY_RDS_FEED,STOCK_CATEGORY_RECOVERED_OIL,STOCK_CATEGORY_RESID,STOCK_CATEGORY_SR_DIESEL,STOCK_CATEGORY_SR_JET,STOCK_CATEGORY_SULFUR,STOCK_CATEGORY_TETRAMER,STOCK_CATEGORY_TRANSMIX,STOCK_CATEGORY_WAXY_BASE_OIL,TRANSIT_TYPE_IN_TRANSIT,TERMINAL_TYPE_REFINERY,TERMINAL_TYPE_TERMINAL,REFINERY_AREA_DESC_EL_SEGUNDO,REFINERY_AREA_DESC_PASADENA,REFINERY_AREA_DESC_PASCAGOULA,REFINERY_AREA_DESC_RICHMOND,REFINERY_AREA_DESC_SALT_LAKE,LOCATION_LEVEL_0_US_EAST,LOCATION_LEVEL_0_US_WEST_COAST,LOCATION_LEVEL_1_IDAHO_TERMINALS,LOCATION_LEVEL_1_LATAM_SYSTEM,LOCATION_LEVEL_1_PASADENA_SUPPLY_AREA,LOCATION_LEVEL_1_PASCAGOULA_SUPPLY_AREA,LOCATION_LEVEL_1_PNW_TERMINALS,LOCATION_LEVEL_1_RICHMOND_SUPPLY_AREA,LOCATION_LEVEL_1_SALT_LAKE_REFINERY,LOCATION_LEVEL_1_SALT_LAKE_TERMINALS,LOCATION_LEVEL_1_SOUTH_TERMINALS,LOCATION_LEVEL_2_BOISE_TERMINAL,LOCATION_LEVEL_2_BURLEY_TERMINAL,LOCATION_LEVEL_2_CEDARCTY,LOCATION_LEVEL_2_COLOMBIA,LOCATION_LEVEL_2_COLONIAL,LOCATION_LEVEL_2_ECENTRAL_AMERICA,LOCATION_LEVEL_2_EL_PASO,LOCATION_LEVEL_2_EL_SEGUNDO_REFINERY,LOCATION_LEVEL_2_FLORIDA,LOCATION_LEVEL_2_LA_MS_OH_TN,LOCATION_LEVEL_2_LVHOLLY,LOCATION_LEVEL_2_NY_HARBOR,LOCATION_LEVEL_2_PANAMA,LOCATION_LEVEL_2_PASADENA_REFINERY,LOCATION_LEVEL_2_PASC_RACK_FOB,LOCATION_LEVEL_2_PASCAGOULA_REFINERY,LOCATION_LEVEL_2_PASCO,LOCATION_LEVEL_2_PASCO_PPL,LOCATION_LEVEL_2_PHOENIX_AREA,LOCATION_LEVEL_2_PLANTATION_DUAL,LOCATION_LEVEL_2_POCATELLO_TERMINAL,LOCATION_LEVEL_2_POCATELLO_UPRR,LOCATION_LEVEL_2_SALT_LAKE_PPL_INTRANSIT,LOCATION_LEVEL_2_SALT_LAKE_REFINERY,LOCATION_LEVEL_2_SALT_LAKE_TERMINAL,LOCATION_LEVEL_2_SOCAL_AREA,LOCATION_LEVEL_2_SPOKANE_HOLLY,LOCATION_LEVEL_2_TEXAS,LOCATION_LEVEL_2_WCENTRAL_AMERICA,TARGET_TYPE_DEFINED,REFINERY_AREA_CA,REFINERY_AREA_ELPA,REFINERY_AREA_ELSE,REFINERY_AREA_GT,REFINERY_AREA_HN,REFINERY_AREA_HNOA,REFINERY_AREA_PA,REFINERY_AREA_PAOA,REFINERY_AREA_PASA,REFINERY_AREA_PASC,REFINERY_AREA_RICH,REFINERY_AREA_SLAK,REFINERY_AREA_SV,YEAR_2022,YEAR_2023,YEAR_2024,MONTH_AUG,MONTH_DEC,MONTH_FEB,MONTH_JAN,MONTH_JUL,MONTH_JUN,MONTH_MAR,MONTH_MAY,MONTH_NOV,MONTH_OCT,MONTH_SEP
1,78.82,223.052917,e7c5a8a2b253d2d195f32439f4e7b9259cc2c8b5f47a07...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,78.82,224.191012,dfc68bbc395067134d5d6e54be9352a65b0d6ba38e2d53...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
13,78.82,221.212898,4459498978e9508cf0dc4071eeb414248aad4efdc2110a...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0

In [85]:
predicted_actual_values =  best_knn.predict(future_input_data.drop(['VOLUME_MBD_ACTUAL','RECORD_ID'], axis=1))
future_input_data['VOLUME_MBD_ACTUAL'] = predicted_actual_values

In [86]:
merged_future_data = pd.merge(original_future_data.drop(['VOLUME_MBD_ACTUAL'], axis=1), future_input_data[['VOLUME_MBD_ACTUAL','RECORD_ID']] , on=['RECORD_ID'], how='left')

#merged_future_data['MONTH'] = pd.to_datetime(merged_future_data['MONTH'], format='%b').dt.month

# Create a new Date column with the last day of the month
#merged_future_data['INVENTORY_DATE'] = pd.to_datetime(merged_future_data[['YEAR', 'MONTH']].assign(day=1)) + pd.offsets.MonthEnd(0)

# Format the date as DD/MM/YYYY
#merged_future_data['INVENTORY_DATE'] = merged_future_data['INVENTORY_DATE'].dt.strftime('%d/%m/%Y')

merged_future_data


,STOCK_FAMILY,STOCK_CATEGORY_NEW,STOCK_CATEGORY,STOCK_SUB_CATEGORY,INVENTORY_DATE,PLANT_NAME,TARGET_TYPE,ITEM_TYPE,TRANSIT_TYPE,TERMINAL_TYPE,REFINERY_AREA_DESC,LOCATION_ID,LOCATION_LEVEL_0,LOCATION_LEVEL_1,LOCATION_LEVEL_2,LOCATION_LEVEL_3,REFINERY_AREA,PLANT_STATUS,VOLUME_MBD_TARGET,RECORD_ID,VOLUME_MBD_ACTUAL
0,CRUDE,CRUDE,CRUDE,CRUDE,2021-10-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,e7c5a8a2b253d2d195f32439f4e7b9259cc2c8b5f47a07...,223.052917
1,CRUDE,CRUDE,CRUDE,CRUDE,2021-11-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,dfc68bbc395067134d5d6e54be9352a65b0d6ba38e2d53...,224.191012
2,CRUDE,CRUDE,CRUDE,CRUDE,2021-12-31,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,4459498978e9508cf0dc4071eeb414248aad4efdc2110a...,221.212898
3,CRUDE,CRUDE,CRUDE,CRUDE,2022-01-30,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,6501692fba7eddfaec805e76bb486092f0003a6015cfb9...,212.713771
4,CRUDE,CRUDE,CRUDE,CRUDE,2022-02-28,MISSISSIPPI MS MAR INTRANSIT,CALCULATED,TargetInventory,IN TANK,IN TRANSIT,PASCAGOULA,81,US EAST,PASCAGOULA SUPPLY AREA,PASCAGOULA REFINERY,PASCAGOULA,PASC,ACTIVE,78.82,c53d8bd441c8dea12bf2bd575699b3f3185107aac76cf5...,214.548868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7827,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WASKOM TX TRM SUNOCO,CALCULATED,TargetInventory,IN TANK,TERMINAL,PASCAGOULA,848,US EAST,PASCAGOULA SUPPLY AREA,FLORIDA,PANAMA CITY,PASC,ACTIVE,4.23,a85bca20d670421cc88c63d4132157a55aeed2443ad166...,5.034015
7828,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WATSON PPL CA INTRANSIT,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,EL SEGUNDO,797,US WEST COAST,EL SEGUNDO SUPPLY AREA,SOCAL AREA,KM SOCAL,ELSE,ACTIVE,107.85,0ab1963368e77af69d8fe7fc0a6d4ece7cd8513d942966...,33.052462
7829,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,WILLIAMS PPL OK INTRANSIT,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,PASCAGOULA,848,US EAST,PASCAGOULA SUPPLY AREA,FLORIDA,PANAMA CITY,PASC,ACTIVE,2.20,d85f3d9c555d3645f8cfc663bc5b37b0095d48d0ee384e...,3.010701
7830,PRODUCTS,GASOLINE,MOGAS,RUL,2024-12-31,YUMBO - BUENAVENTURA PPL,CALCULATED,TargetInventory,IN TRANSIT,IN TRANSIT,CENTRAL AMERICA,478,US EAST,LATAM SYSTEM,COLOMBIA,COLOMBIA,CA,ACTIVE,8.48,8d0e2dc85623a47298eb31cd44f078c0e498fdde03c0c0...,12.099508


In [87]:
merged_future_data.to_csv('predicted_future_data.csv')